<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Betting%20Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Import Sportsbook data for the 2019-2020 season.
nhl_lines_1920_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Odds%20Data/nhl%20odds%202019-20.csv')
#nhl_lines_1920_raw.head()

In [3]:
#Reduce data for CLV of Sportsbooks
nhl_lines_1920 = nhl_lines_1920_raw.loc[:,['Date','Team','Close']]
#nhl_lines_1920.head()

In [4]:
#Change Close column to float types
nhl_lines_1920.Close.astype(float)

0       270.0
1      -310.0
2       135.0
3      -150.0
4       116.0
        ...  
2419    135.0
2420    150.0
2421   -165.0
2422   -170.0
2423    155.0
Name: Close, Length: 2424, dtype: float64

In [5]:
#Make a column for the vig of all teams

#Make a condition for when Underdog Odds
nhl_lines_1920.loc[nhl_lines_1920['Close'] >= 0, 'Win %'] = 1/((nhl_lines_1920['Close']/100)+1)
#Make a condition for when Favorite odds
nhl_lines_1920.loc[nhl_lines_1920['Close'] <= 0, 'Win %'] = 1/(1+(-100/nhl_lines_1920['Close']))

nhl_lines_1920

,Date,Team,Close,Win %
0,1002,Ottawa,270,0.270270
1,1002,Toronto,-310,0.756098
2,1002,Washington,135,0.425532
3,1002,St.Louis,-150,0.600000
4,1002,Vancouver,116,0.462963
...,...,...,...,...
2419,925,Dallas,135,0.425532
2420,926,Dallas,150,0.400000
2421,926,TampaBay,-165,0.622642
2422,928,TampaBay,-170,0.629630


In [6]:
#Add column for Log Loss
nhl_lines_1920['Log Loss'] = -np.log(1-(1-nhl_lines_1920['Win %']))

In [7]:
#Make a table for betting lines for each team during the 2019-2020 season.
Avalanche_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Colorado']
Blackhawks_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Chicago']
Blue_Jackets_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Columbus']
Blues_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='St.Louis']
Bruins_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Boston']
Canadiens_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Montreal']
Canucks_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Vancouver']
Capitals_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Washington']
Coyotes_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Arizona']
Devils_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='NewJersey']
Ducks_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Anaheim']
Flames_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Calgary']
Flyers_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Philadelphia']
Hurricanes_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Carolina']
Islanders_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='NYIslanders']
Jets_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Winnipeg']
Kings_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='LosAngeles']
Knights_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Vegas']
Leafs_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Toronto']
Lightning_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='TampaBay']
Oilers_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Edmonton']
Panthers_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Florida']
Penguins_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Pittsburgh']
Predators_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Nashville']
Rangers_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='NYRangers']
Red_Wings_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Detroit']
Sabres_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Buffalo']
Senators_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Ottawa']
Sharks_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='SanJose']
Stars_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Dallas']
Wild_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Minnesota']

In [9]:
#Reset index for Team tables
Avalanche_1920.reset_index(inplace=True)
Blackhawks_1920.reset_index(inplace=True)
Blue_Jackets_1920.reset_index(inplace=True)
Blues_1920.reset_index(inplace=True)
Bruins_1920.reset_index(inplace=True)
Canadiens_1920.reset_index(inplace=True)
Canucks_1920.reset_index(inplace=True)
Capitals_1920.reset_index(inplace=True)
Coyotes_1920.reset_index(inplace=True)
Devils_1920.reset_index(inplace=True)
Ducks_1920.reset_index(inplace=True)
Flames_1920.reset_index(inplace=True)
Flyers_1920.reset_index(inplace=True)
Hurricanes_1920.reset_index(inplace=True)
Islanders_1920.reset_index(inplace=True)
Jets_1920.reset_index(inplace=True)
Kings_1920.reset_index(inplace=True)
Knights_1920.reset_index(inplace=True)
Leafs_1920.reset_index(inplace=True)
Lightning_1920.reset_index(inplace=True)
Oilers_1920.reset_index(inplace=True)
Panthers_1920.reset_index(inplace=True)
Penguins_1920.reset_index(inplace=True)
Predators_1920.reset_index(inplace=True)
Rangers_1920.reset_index(inplace=True)
Red_Wings_1920.reset_index(inplace=True)
Sabres_1920.reset_index(inplace=True)
Senators_1920.reset_index(inplace=True)
Sharks_1920.reset_index(inplace=True)
Stars_1920.reset_index(inplace=True)
Wild_1920.reset_index(inplace=True)

In [10]:
#Restrict data to just regular season games.
Avalanche_1920.iloc[:70]
Blackhawks_1920.iloc[:70]

,index,Date,Team,Close,Win %,Log Loss
0,24,1004,Chicago,107,0.483092,0.727549
1,107,1010,Chicago,100,0.500000,0.693147
2,137,1012,Chicago,-130,0.565217,0.570545
3,167,1014,Chicago,-110,0.523810,0.646627
4,219,1018,Chicago,-135,0.574468,0.554311
...,...,...,...,...,...,...
65,2049,303,Chicago,-180,0.642857,0.441833
66,2077,305,Chicago,-107,0.516908,0.659890
67,2084,306,Chicago,-190,0.655172,0.422857
68,2121,308,Chicago,120,0.454545,0.788457


In [11]:
Blackhawks_1920

,index,Date,Team,Close,Win %,Log Loss
0,24,1004,Chicago,107,0.483092,0.727549
1,107,1010,Chicago,100,0.500000,0.693147
2,137,1012,Chicago,-130,0.565217,0.570545
3,167,1014,Chicago,-110,0.523810,0.646627
4,219,1018,Chicago,-135,0.574468,0.554311
...,...,...,...,...,...,...
74,2256,811,Chicago,180,0.357143,1.029619
75,2270,813,Chicago,190,0.344828,1.064711
76,2293,815,Chicago,145,0.408163,0.896088
77,2299,816,Chicago,175,0.363636,1.011601


In [12]:
#Restrict Avalanche data to just regular season games.
Blackhawks_1920.iloc[:70]

,index,Date,Team,Close,Win %,Log Loss
0,24,1004,Chicago,107,0.483092,0.727549
1,107,1010,Chicago,100,0.500000,0.693147
2,137,1012,Chicago,-130,0.565217,0.570545
3,167,1014,Chicago,-110,0.523810,0.646627
4,219,1018,Chicago,-135,0.574468,0.554311
...,...,...,...,...,...,...
65,2049,303,Chicago,-180,0.642857,0.441833
66,2077,305,Chicago,-107,0.516908,0.659890
67,2084,306,Chicago,-190,0.655172,0.422857
68,2121,308,Chicago,120,0.454545,0.788457
